In [44]:
import pandas as pd
import requests
import base64

In [45]:
pd.set_option('display.max_colwidth', 0)

In [46]:
pat = ""
authorization = str(base64.b64encode(bytes(':'+pat, 'ascii')), 'ascii')

headers = {
 'Accept': 'application/json',
 'Authorization': 'Basic '+authorization
}


In [47]:
df = pd.read_csv("us_cities.csv")

In [48]:
df.head()

,ID,STATE_CODE,STATE_NAME,CITY,COUNTY,LATITUDE,LONGITUDE
0,1,AK,Alaska,Adak,Aleutians West,55.999722,-161.207778
1,2,AK,Alaska,Akiachak,Bethel,60.891854,-161.392330
2,3,AK,Alaska,Akiak,Bethel,60.890632,-161.199325
3,4,AK,Alaska,Akutan,Aleutians East,54.143012,-165.785368
4,5,AK,Alaska,Alakanuk,Kusilvak,62.746967,-164.602280


,ID,STATE_CODE,STATE_NAME,CITY,COUNTY,LATITUDE,LONGITUDE,URL
1684,1685,CA,California,Acampo,San Joaquin,38.200413,-121.218576,"https://api.geoapify.com/v2/places?categories=tourism,entertainment,catering.restaurant&filter=circle:-121.218576,38.200413,1000&bias=proximity:-121.218576,38.200413&limit=500&apiKey=c8a825d7da834647b40102fcad6d5b3f"
1685,1686,CA,California,Acton,Los Angeles,34.483541,-118.195929,"https://api.geoapify.com/v2/places?categories=tourism,entertainment,catering.restaurant&filter=circle:-118.195929,34.483541,1000&bias=proximity:-118.195929,34.483541&limit=500&apiKey=c8a825d7da834647b40102fcad6d5b3f"
1686,1687,CA,California,Adelanto,San Bernardino,34.584128,-117.424189,"https://api.geoapify.com/v2/places?categories=tourism,entertainment,catering.restaurant&filter=circle:-117.424189,34.584128,1000&bias=proximity:-117.424189,34.584128&limit=500&apiKey=c8a825d7da834647b40102fcad6d5b3f"
1687,1688,CA,California,Adin,Modoc,41.217510,-120.943193,"https://api.geoapify.com/v2/places?categories=tourism,entertainment,catering.restaurant&filter=circle:-120.943193,41.21751,1000&bias=proximity:-120.943193,41.21751&limit=500&apiKey=c8a825d7da834647b40102fcad6d5b3f"
1688,1689,CA,California,Agoura Hills,Los Angeles,34.157163,-118.757180,"https://api.geoapify.com/v2/places?categories=tourism,entertainment,catering.restaurant&filter=circle:-118.75718,34.157163,1000&bias=proximity:-118.75718,34.157163&limit=500&apiKey=c8a825d7da834647b40102fcad6d5b3f"
...,...,...,...,...,...,...,...,...
2921,2922,CA,California,Yuba City,Sutter,39.128619,-121.621599,"https://api.geoapify.com/v2/places?categories=tourism,entertainment,catering.restaurant&filter=circle:-121.621599,39.128619,1000&bias=proximity:-121.621599,39.128619&limit=500&apiKey=c8a825d7da834647b40102fcad6d5b3f"
2922,2923,CA,California,Yucaipa,San Bernardino,34.028197,-117.048925,"https://api.geoapify.com/v2/places?categories=tourism,entertainment,catering.restaurant&filter=circle:-117.048925,34.028197,1000&bias=proximity:-117.048925,34.028197&limit=500&apiKey=c8a825d7da834647b40102fcad6d5b3f"
2923,2924,CA,California,Yucca Valley,San Bernardino,34.155936,-116.431313,"https://api.geoapify.com/v2/places?categories=tourism,entertainment,catering.restaurant&filter=circle:-116.431313,34.155936,1000&bias=proximity:-116.431313,34.155936&limit=500&apiKey=c8a825d7da834647b40102fcad6d5b3f"
2924,2925,CA,California,Zamora,Yolo,38.799896,-121.906540,"https://api.geoapify.com/v2/places?categories=tourism,entertainment,catering.restaurant&filter=circle:-121.90654,38.799896,1000&bias=proximity:-121.90654,38.799896&limit=500&apiKey=c8a825d7da834647b40102fcad6d5b3f"


In [49]:
def form_url(row):
    
    long = row.LONGITUDE
    lat = row.LATITUDE
    
    url = "https://api.geoapify.com/v2/places?categories=tourism,entertainment,catering.restaurant&filter=circle:{},{},1000&bias=proximity:{},{}&limit=500&apiKey=c8a825d7da834647b40102fcad6d5b3f".format(long,lat,long,lat)
    return url


In [50]:
df['URL'] = df.apply (lambda row: form_url(row), axis=1)


In [51]:
df.iloc[0].URL

'https://api.geoapify.com/v2/places?categories=tourism,entertainment,catering.restaurant&filter=circle:-161.207778,55.999722,1000&bias=proximity:-161.207778,55.999722&limit=500&apiKey=c8a825d7da834647b40102fcad6d5b3f'

In [52]:
df[df['CITY']=="New York"]

,ID,STATE_CODE,STATE_NAME,CITY,COUNTY,LATITUDE,LONGITUDE,URL
18874,18875,NY,New York,New York,New York,40.74838,-73.996705,"https://api.geoapify.com/v2/places?categories=tourism,entertainment,catering.restaurant&filter=circle:-73.996705,40.74838,1000&bias=proximity:-73.996705,40.74838&limit=500&apiKey=c8a825d7da834647b40102fcad6d5b3f"


In [53]:
top_cities = ["New York", "Los Angeles", "Las Vegas", "Orlando", "Miami", "San Francisco"]

In [54]:
df_top = df[df['CITY'].isin(top_cities)]

In [56]:
df_top = df_top.reset_index()  # make sure indexes pair with number of rows


In [59]:
for index, row in df_top.iterrows():
    
    city = row["CITY"]
    state = row["STATE_CODE"]
    county = row["COUNTY"]
    url = row["URL"]
    
    filename = city+'-'+state+'.json'
    
    response = requests.get(url, headers=headers)

    with open(filename, "wb") as text_file:
        text_file.write(response.content)